In [2]:
import numpy as np
import os
import math
from collections import defaultdict
import scipy.spatial.distance as dist
os.chdir("../../Data/1317_ddi_chem")

In [2]:
def get_new_chem(chem):
    degree = np.sum(chem,axis=0)
    index_mapping = {}
    i = 0
    for index in range(len(degree)):
        if not degree[index] == 0:
            index_mapping[i] =index
            i += 1

    buffer = chem.T
    new_chem = []
    for index in range(buffer.shape[0]):
        if index in index_mapping.values():
            new_chem.append(buffer[index])
    new_chem = np.array(new_chem).T
    return new_chem,index_mapping

def get_w_new_chem(new_chem,index_mapping):
    degree = np.sum(new_chem,axis=0)
    assert not min(degree) == 0
    w_new_chem = []
    for index in range(len(new_chem.T)):
        w_new_chem.append(new_chem.T[index]/degree[index])
    w_new_chem = np.array(w_new_chem).T
    return w_new_chem

def get_inter_f(binary,new_chem):
    #利用tf-idf计算权重值
    #idf：log10 inter_count/(fre+1)
    #接着计算药物的反应药物集合中特定结构出现的 次数/总次数 值
    total = 0
    inter_count =0
    inter_f = []
    total_buffer = np.zeros((new_chem.shape[1]))
    for i in range(len(binary)):
        total = 0
        buffer = np.zeros((new_chem.shape[1]))
        for j in range(len(binary)):
            if binary[i][j] == 1:
                total = sum(new_chem[j])
                buffer = buffer + new_chem[j]#获取反应药物的子结构特征数量和
                inter_count += 1
        tf = buffer/total
        total_buffer = total_buffer + buffer
        inter_f.append(tf)
        if i% 200 == 0:
            print("%d/%d"%(i,len(binary)))
    #idf
    fre = total_buffer
    idf = np.log10(inter_count/(fre+1))
    inter_f = [i*idf for i in inter_f]
    inter_f = np.array(inter_f)
    inter_f_ravel = inter_f.ravel()
    #减小数据偏差，大于10的数据值设置为10
    for index in range(len(inter_f_ravel)):
        if inter_f_ravel[index]>10:
            inter_f_ravel[index] = 10.
    inter_f = inter_f_ravel.reshape(inter_f.shape[0],inter_f.shape[1])
    return inter_f  

def chem_type(i):
    if i>=0 and i<=114:
        return str(0)
    elif i>=115 and i<=262:
        return str(1)
    elif i>=263 and i<=326:
        return str(2)
    elif i>=327 and i<=415:
        return str(3)
    elif i>=416 and i<=459:
        return str(4)
    elif i>=460 and i<=712:
        return str(5)
    elif i>=713 and i<=880:
        return str(6)
    else:
        print('exception')
        return -1

In [14]:
data_1317 = np.load('1317_drug_v5.npz',allow_pickle=True)
data_1317 = data_1317['data'].item()
chem = data_1317["Drug_chem_structure"]
binary = data_1317["Adj_binary"]
new_chem,index_mapping = get_new_chem(chem)#new_chem去除了度为0的化学结构，index_mapping是683-》881的映射
# w_new_chem = get_w_new_chem(new_chem,index_mapping)
# inter_f  = get_inter_f(binary,new_chem)#药物相互作用药物的特征，权重
w_f = data_1317['interaction_feature']

In [5]:
#数据的重新生成  kg_final.txt  权重保留0.05的边加入到网络当中
# 药物 类别 属性
with open("kg_final.txt",'w') as f:
    index = 0
    for drug in w_f:
        # 对当前的特征进行排序(前150个子结构)
        sort_index = np.argsort(drug)[-1::-1][0:150]
        for i in range(len(drug)):
            if w_f[index][i] >= 0.05 and i in sort_index:
                f.writelines(str(index) +' '+chem_type(index_mapping[i])+' '+str(i+1317)+'\n')
        index += 1


In [42]:
a = w_f[0]
index_sort1= np.argsort(a)[-1::-1]
print(index_sort1[0:10])
print(a[index_sort1][0:10])

[254 265 199 425 362 224 478 143 511 491]
[0.6380728  0.59357701 0.56149264 0.53990657 0.49121346 0.485607
 0.48031382 0.47882695 0.47089431 0.46180015]


In [43]:
a = w_f[1]
index_sort2= np.argsort(a)[-1::-1]
print(index_sort2[0:10])
print(a[index_sort2][0:10])

[337 374 466 211 450 159 419 377 194 263]
[1.00897034 0.98573759 0.96274278 0.90382602 0.90130397 0.85822297
 0.83775707 0.8352062  0.8309763  0.81967037]


In [60]:
len(set(index_sort1[0:100]) & set(index_sort4[0:100]))

41

In [45]:
a = w_f[2]
index_sort3= np.argsort(a)[-1::-1]
print(index_sort3[0:10])
print(a[index_sort3][0:10])

[254 199 265  16 478 425 224 362 143 479]
[0.84335441 0.79188331 0.78193693 0.76941087 0.7518707  0.74417666
 0.72535119 0.71659802 0.71494359 0.6905088 ]


In [59]:
a = w_f[777]
index_sort4= np.argsort(a)[-1::-1]
print(index_sort4[0:10])
print(a[index_sort4][0:10])

[ 12 123 178 120 206 461 369 194 215 221]
[0.42970311 0.42549825 0.39880929 0.39849377 0.39787523 0.39631833
 0.39543901 0.39211006 0.39193495 0.39082993]


In [41]:
import scipy.sparse as sp
#稀疏矩阵的构建，其中包括药物之间的相互作用权重（1）和药物属性之间的权重
#一个小demo
# ddi_matrix = np.array([[0,1,0,1],[1,0,1,0],[0,1,0,1],[1,0,1,0]])
# dfi_matrix = np.array([[0.3,0.5,0.1],[0.1,0.1,0.7],[0.6,0.4,0.4],[0.5,0.2,0.2]])
def get_w_sp_matrix(ddi_matrix,dfi_matrix):
    #药物个数，特征个数
    n_drug = ddi_matrix.shape[0]
    n_feature = dfi_matrix.shape[1]
    #先转稀疏矩阵
    ddi_sp = sp.coo_matrix(ddi_matrix)
    dfi_sp = sp.coo_matrix(dfi_matrix)
    #ddi的行列
    ddi_row = ddi_sp.row
    ddi_col = ddi_sp.col+n_drug
    #dfi的行列
    dfi_row = dfi_sp.row + n_drug
    dfi_col = dfi_sp.col +2*n_drug
    #最终的行列
    rows =  np.concatenate((ddi_row,dfi_row))
    cols =  np.concatenate((ddi_col,dfi_col))
    values = np.concatenate((ddi_sp.data,dfi_sp.data))
    #
    result = sp.coo_matrix((values, (rows, cols)), shape=(n_drug*2 + n_feature, n_drug*2 + n_feature))
    return result

In [42]:
result = get_w_sp_matrix(binary,inter_f)

In [45]:
result.toarray()[0][1300:1400]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [49]:
index = ['Drug_id',
 'Num_of_type2',
 'All_proteins',
 'Adj_binary',
 'Drug_chem_structure',
 'Drug_protein',
 'Sim_chem_by_jaccard',
 'Adj_multi2',
 'interaction_feature'
 ]

In [50]:
new_data = {"Drug_id":data_1317["Drug_id"],'Num_of_type2':86,'All_proteins':data_1317["All_proteins"],
           'Adj_binary':data_1317["Adj_binary"],'Adj_multi2':data_1317["Adj_multi2"],
           'Drug_chem_structure':data_1317["Drug_chem_structure"],'interaction_feature':inter_f,
           'Drug_protein':data_1317["Drug_protein"],'Sim_chem_by_jaccard':data_1317["Sim_chem_by_jaccard"],'index':index}
np.savez('1317_drug_v5.npz',data = new_data)

In [2]:
data_1317 = np.load('1317_drug_v4.npz',allow_pickle=True)
data_1317 = data_1317['data'].item()
chem = data_1317["Drug_chem_structure"]
binary = data_1317["Adj_binary"]

In [3]:
def get_train_adj():
    adj = np.zeros((1317,1317))
    with open('train.txt') as f:
        line = f.readline()
        while line:
            line = list(map(int,line.split(' ')[:-1]))
            drug1 = line[0]
            drug2 = line[1:]
            for d in drug2:
                adj[drug1,d] = adj[d,drug1] = 1
            line = f.readline()
    return adj
adj = get_train_adj()

In [4]:
def get_inter_sim(binary):
    inter_sim = np.zeros((1317,1317))
    for row in range(len(binary)):
        for column in range(row,len(binary)):
            matv = np.array([binary[row],binary[column]])
            inter_sim[row,column] = inter_sim[column,row] = math.exp(-(dist.pdist(matv,'jaccard')))  
        if row %20 == 0:
            print(row)
    return inter_sim
binary = data_1317["Adj_binary"]
inter_sim = get_inter_sim(binary)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300


In [28]:
inter_sim

array([[1.        , 0.40063658, 0.46653651, ..., 0.3806186 , 0.39930378,
        0.37288453],
       [0.40063658, 1.        , 0.46891724, ..., 0.37132559, 0.38338499,
        0.37056795],
       [0.46653651, 0.46891724, 1.        , ..., 0.3862939 , 0.38145282,
        0.37974551],
       ...,
       [0.3806186 , 0.37132559, 0.3862939 , ..., 1.        , 0.36787944,
        0.37416783],
       [0.39930378, 0.38338499, 0.38145282, ..., 0.36787944, 1.        ,
        0.36787944],
       [0.37288453, 0.37056795, 0.37974551, ..., 0.37416783, 0.36787944,
        1.        ]])

In [5]:
#某一对负样本的可信度
def get_reliable_negative(d1,d2):
    sim1,sim2 = 0.,0.
    #d2与d1的反应药物之间的相似度
    for index in range(len(adj[d1])):
        if adj[d1][index] == 1:
            sim1 += inter_sim[d2][index]
    for index in range(len(adj[d2])):
        if adj[d2][index] == 1:
            sim2 += inter_sim[d1][index]
    return math.exp(-(sim1+sim2))

In [6]:
reliable_negative = defaultdict(list)
for row in range(1317):
    for column in range(row,1317):
        if adj[row,column] == 0:
            neg_score = get_reliable_negative(row,column)
            reliable_negative[row].append((column,neg_score))
    if row %10 == 0:
        print(row)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310


In [7]:
reliable_negative_m = defaultdict(list)
for k,value in reliable_negative.items():
    for v in value:
        reliable_negative_m[k].append((v[1],v[0]))
        reliable_negative_m[v[0]].append((v[1],k))

In [8]:
reliable_negative_dict = {}
for k,value in reliable_negative_m.items():
    reliable_neg = sorted(list(set(value)))
    reliable_negative_dict[k] = reliable_neg[-1::-1]

In [9]:
# reliable_negative_dict[0][0:200]
new_data = {"reliable_negative_dict":reliable_negative_dict}
np.savez('reliable_negative_dict.npz',data = new_data)

In [10]:
negative = np.load('reliable_negative_dict.npz',allow_pickle=True)
negative = negative['data'].item()
negative_dict = negative["reliable_negative_dict"]

In [11]:
len(negative_dict[2])

1092